<a href="https://colab.research.google.com/github/AniluG03/AI-Fracture-Classification-/blob/main/AI-Fracture%20(without%20results).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Redes Neuronales**

Actividad: PIA

Ing. Tomás Eloy Salais Fierro

Integrantes:

Lucía Garza Garza 1956237

Jorge Eduardo Muñoz Garza 2077981

Ana Luisa Garza Díaz 1948335


#Instalación de librerías.

In [ ]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import datasets, layers, models, utils, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#Descarga de datos del repositorio de Kaggle.

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d pkdarabi/bone-break-classification-image-dataset

In [ ]:
!unzip -o "bone-break-classification-image-dataset.zip"

In [ ]:
!ls

In [ ]:
!kaggle datasets download -d devbatrax/fracture-detection-using-x-ray-images

In [ ]:
!ls

In [ ]:
!unzip -o "fracture-detection-using-x-ray-images.zip"

In [ ]:
!mkdir -p "/content/Bone Break Classification/Bone Break Classification/No Fracture"  # Crear carpeta

In [ ]:
!mv "/content/data/train/not fractured" "/content/Bone Break Classification/Bone Break Classification/No Fracture"

In [ ]:
!mv "/content/Bone Break Classification/Bone Break Classification/No Fracture/not fractured" "/content/Bone Break Classification/Bone Break Classification/No Fracture/Train"  # Renombrar

In [ ]:
!mv "/content/data/val/not fractured" "/content/Bone Break Classification/Bone Break Classification/No Fracture"

In [ ]:
!mv "/content/Bone Break Classification/Bone Break Classification/No Fracture/not fractured" "/content/Bone Break Classification/Bone Break Classification/No Fracture/Test"  # Renombrar

In [ ]:
#!rm -r "/content/data/train/carpeta_a_eliminar"  # Eliminar

# Cargar y etiquetar imágenes

In [ ]:
data_dir = "/content/Bone Break Classification/Bone Break Classification"


#Cargar imagenes de entrenamiento y validación
train_data = utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="training",
    shuffle=True,
    color_mode="rgb",
    image_size=(100,100),
    batch_size=64,
    seed=40
)


validation_data = utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="validation",
    color_mode="rgb",
    image_size=(100,100),
    batch_size=64,
    seed=42
)

In [ ]:
# Buscar clases
class_names = train_data.class_names
print("Class names:", class_names)

In [ ]:
# Normalización
def preprocess(img):
    img = tf.cast(img, "float32") / 255.0
    return img

train_dataset = train_data.map(lambda x, y: (preprocess(x), y))
val_dataset = validation_data.map(lambda x, y: (preprocess(x), y))

Acomodo de etiquetas e imagenes en arreglos

In [ ]:
x_train = []
y_train = []

for images, labels in train_dataset:
    #append train data to x_train
    x_train.append(images.numpy())

    #append labels to y_train
    y_train.append(labels.numpy())

x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

In [ ]:
x_val = []
y_val = []

for images, labels in val_dataset:
    #append train data to x_train
    x_val.append(images.numpy())

    #append labels to y_train
    y_val.append(labels.numpy())


x_val = np.concatenate(x_val, axis=0)
y_val = np.concatenate(y_val, axis=0)

In [ ]:
print(len(x_val))

In [ ]:
# Definición de clases
class_names_dict = {
    0: 'Avulsion fracture',
    1: 'Comminuted fracture',
    2: 'Fracture Dislocation',
    3: 'Greenstick fracture',
    4: 'Hairline Fracture',
    5: 'Impacted fracture',
    6: 'Longitudinal fracture',
    7: 'No fracture',
    8: 'Oblique fracture',
    9: 'Pathological fracture',
    10: 'Spiral Fracture'
}

Comprobación de imagenes

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_val[i], cmap='gray')

    plt.xlabel(class_names[y_val[i]])
plt.show()

In [ ]:
x_train.shape

In [ ]:
NUM_CLASSES = len(class_names)

train_labels = utils.to_categorical(y_train, NUM_CLASSES)
test_labels = utils.to_categorical(y_val, NUM_CLASSES)

#Construir modelo de la red neuronal

In [ ]:

# Construcción del modelo
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(momentum=0.9),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(momentum=0.9),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(momentum=0.9),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    layers.BatchNormalization(momentum=0.9),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(momentum=0.9),
    layers.Dropout(0.5),

    layers.Dense(len(class_names), activation='softmax')
])

# Compilar el modelo
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]



In [ ]:
model.summary()

In [ ]:
# Entrenar el modelo sin aumento de datos
history = model.fit( x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val), callbacks=callbacks)

# Evaluar el modelo
test_loss, test_acc = model.evaluate(x_val, y_val)
print('Test accuracy:', test_acc)

# Precisión y Pérdida: Entrenamiento VS Validación

In [ ]:
# Graficar la precisión y la pérdida durante el entrenamiento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Precisión en Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión Validada')
plt.legend(loc='lower right')
plt.title('Precisión de Entrenamiento y Validación')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en Validación')
plt.legend(loc='upper right')
plt.title('Pérdida en Entrenamiento y Validación')
plt.show()

# Predicciones

In [ ]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [ ]:
results = probability_model.predict(x_val)

In [ ]:
results[3]

Otra forma

In [ ]:
predictions = model.predict(x_val)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(11))
  plt.yticks([])
  thisplot = plt.bar(range(11), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 2


plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], y_val, x_val)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  y_val)
plt.show()

In [ ]:
num_rows = 5
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], y_val, x_val)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], y_val)
plt.tight_layout()
plt.show()